In [13]:
! pip list | grep "torch"

torch                    1.5.1+cu101    
torchsummary             1.5.1          
torchtext                0.3.1          
torchvision              0.6.1+cu101    


In [9]:
import re
import sys
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets

## Reading Data

In [3]:

# Data Setting
TEXT = data.Field(batch_first = True,
                  fix_length = 500,
                  tokenize=str.split,
                  pad_first=True,
                  pad_token='[PAD]',
                  unk_token='[UNK]')

LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(text_field = TEXT, 
                                             label_field = LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.2MB/s]


In [4]:
# Data Length
print(f'Train Data Length : {len(train_data.examples)}')
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 25000
Test Data Length : 25000


In [5]:
# Data Fields
train_data.fields

{'label': <torchtext.data.field.LabelField at 0x7f5679fcab38>,
 'text': <torchtext.data.field.Field at 0x7f5679fcabe0>}

In [6]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']),'\n')
print('Label : ')
print(vars(train_data.examples[1])['label'])

---- Data Sample ----
Input : 
Depardieu's most notorious film is this (1974)groundbreaker from Bertrand Blier. It features many highly sexual scenes verging on an X-rating, including one of Jeanne Moreau doing a hot 1970s version of her Jules and Jim menage a trois with the two hairy French hippies (Depardieu and Deware). There is no such thing as a sacred territory in this film; everything is fair game.<br /><br />It's very odd that Americans tend to not like this film very much while many French people I've met consider it a classic. Something about it goes against what Americans have been programmed to 'like.'<br /><br />Gerard and the late Patrick Deware are two bitch-slapping, hippy drifters with many sexual insecurities, going around molesting women and committing petty crimes. They're out for kicks and anti-capitalist, Euro-commie, slacker 'freedom.' Blier satirizes the hell out of these two guys while at the same time making bourgeois society itself look ultimately much more r

## Pre-processing Data

In [7]:
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

In [10]:
for example in train_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
    
for example in test_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

## Making Vocab & Setting Embedding

In [11]:
model_config = {'emb_type' : '', 'emb_dim' : 300}

In [14]:
# no pre-trained 
TEXT.build_vocab(train_data,
                 min_freq = 2, 
                 max_size = None,
                 )

LABEL.build_vocab(train_data)

model_config['vocab_size'] = len(TEXT.vocab)

In [15]:
# Vocabulary Info
print(f'Vocab Size : {len(TEXT.vocab)}')

print('Vocab Examples : ')
for idx, (k, v) in enumerate(TEXT.vocab.stoi.items()):
    if idx >= 10:
        break    
    print('\t', k, v)

print('---------------------------------')

# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)

Vocab Size : 51956
Vocab Examples : 
	 [UNK] 0
	 [PAD] 1
	 the 2
	 and 3
	 a 4
	 of 5
	 to 6
	 is 7
	 in 8
	 it 9
---------------------------------
Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


## Spliting Validation Data & Making Data Iterator

In [16]:
# Spliting Valid set
train_data, valid_data = train_data.split(random_state = random.seed(0),
                                          split_ratio=0.8)

In [17]:
model_config['batch_size'] = 30

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(datasets=(train_data, valid_data, test_data), 
                                                                           batch_size=model_config['batch_size'],
                                                                           device=device)

In [ ]:
# Check batch data
sample_for_check = next(iter(train_iterator))
print(sample_for_check)
print(sample_for_check.text)
print(sample_for_check.label)


[torchtext.data.batch.Batch of size 30]
	[.text]:[torch.cuda.LongTensor of size 30x500 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 30 (GPU 0)]
tensor([[   1,    1,    1,  ...,   43,    5,  155],
        [   1,    1,    1,  ...,   25,    6,  132],
        [   1,    1,    1,  ...,   40, 1041,   75],
        ...,
        [   1,    1,    1,  ...,    5,   65, 1258],
        [ 100,  462,   58,  ...,    3,  148,   56],
        [   1,    1,    1,  ...,  979,  719,  123]], device='cuda:0')
tensor([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.], device='cuda:0')


In [ ]:
# Check reverting data
print(' '.join([TEXT.vocab.itos[int(x)] for x in sample_for_check.text[0,:] if x not in [0,1]]))
print(LABEL.vocab.itos[int(sample_for_check.label[0])]) 

i'm a fan of both shakespeare and mst3k so i waited anxiously to see this episode i'll comment on the movie first then the mst3k episode the recipe for this movie take talented actors rich and beautiful shakespeare material and a 1 25 budget mix well then drain of all life and movement until dull and lifeless serve cold in a big plain stone cauldron movie i give 3 out of 10 because the actors at least deserve a little bit of credit okay now the mst3k episode i'll admit it the first time i saw it i fell asleep halfway through i understand that was the reaction of several other as well however when i watched it a second time i realized that there was a whole host of intelligent references and good lines i missed the first time around the trick with this episode is listen carefully it takes a couple of viewings to catch each line give it a second chance and you'll see what i mean mst3k episode 7 1 2 out of 10
neg


## Making Model

In [ ]:
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()

        if model_config['emb_type'] == 'glove' or 'fasttext':
            self.emb = nn.Embedding(num_embeddings = model_config['vocab_size'],
                                    embedding_dim = model_config['emb_dim'],
                                    _weight = TEXT.vocab.vectors)
        else:
            self.emb = nn.Embedding(num_embeddings = model_config['vocab_size'],
                                    embedding_dim = model_config['emb_dim'])
        
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type'] 

        self.RNN = nn.RNN (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.LSTM= nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.GRU = nn.GRU (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
    
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction,
                            model_config['output_dim'])
        
        self.drop = nn.Dropout(model_config['dropout'])

    def forward(self, x):
        
        emb = self.emb(x) 
        # emb : (Batch_Size, Max_Seq_Length, Emb_dim)

        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb) 
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')
        
        # output : (Batch_Size, Max_Seq_Length, Hidden_dim * num_direction) 
        # hidden : (num_direction, Batch_Size, Hidden_dim)
        # hidden의 경우, batch_first 옵션이 안먹는 문제가 있음
        
        last_output = output[:,-1,:]

        # last_output : (Batch_Size, Hidden_dim * num_direction)
        return self.fc(self.drop(last_output))

### Checking feed-forward

In [ ]:
model_config.update(dict(batch_first = True,
                         model_type = 'RNN',
                         bidirectional = True,
                         hidden_dim = 128,
                         output_dim = 1,
                         dropout = 0))

In [ ]:
model = SentenceClassification(**model_config).to(device)

In [ ]:
predictions = model.forward(sample_for_check.text).squeeze()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss().to(device)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
loss = loss_fn(predictions, sample_for_check.label)
acc = binary_accuracy(predictions, sample_for_check.label)

In [ ]:
print(predictions)
print(loss, acc)

tensor([ 0.5048,  0.4538,  0.2271,  0.0303,  0.3336,  0.0039,  0.1705, -0.5562,
        -0.4903,  0.2203, -0.1409, -0.1375,  0.3909,  0.5392,  0.4122,  0.1316,
        -0.4174, -0.1250,  0.4200, -0.3405,  0.3079,  0.7391, -0.2441,  0.1625,
        -0.0331, -0.1079,  0.1159, -0.7932,  0.0254,  0.4805], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
tensor(0.6816, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) tensor(0.6000, device='cuda:0')


## Training

In [ ]:
def train(model, iterator, optimizer, loss_fn, idx_epoch, **model_params):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    batch_size = model_params['batch_size']

    for idx, batch in enumerate(iterator):
        
        # Initializing
        optimizer.zero_grad()
        
        # Forward 
        predictions = model(batch.text).squeeze()
        loss = loss_fn(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        sys.stdout.write(
                    "\r" + f"[Train] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )

        # Backward 
        loss.backward()
        optimizer.step()
        
        # Update Epoch Performance
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator) , epoch_acc/len(iterator) 

In [ ]:
def evaluate(model, iterator, loss_fn):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # evaluation mode
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = loss_fn(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### bi-RNN

In [ ]:
model_config['model_type'] = 'RNN'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-RNN_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.5298  Acc : 0.7667
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6392 | Train Acc : 0.6268
	 Epoch : 0 | Valid Loss : 0.5983 | Valid Acc : 0.6703
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.6271  Acc : 0.6
	 Epoch : 1 | Train Loss : 0.5281 | Train Acc : 0.7383
	 Epoch : 1 | Valid Loss : 0.6394 | Valid Acc : 0.6211
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.4983  Acc : 0.8
	 Epoch : 2 | Train Loss : 0.4999 | Train Acc : 0.7573
	 Epoch : 2 | Valid Loss : 0.6128 | Valid Acc : 0.6722
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.2764  Acc : 0.8667
	 Epoch : 3 | Train Loss : 0.445 | Train Acc : 0.7922
	 Epoch : 3 | Valid Loss : 0.6013 | Valid Acc : 0.7018
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.4749  Acc : 0.8333
	 Epoch : 4 | Train Loss : 0.3589 | Train Acc : 0.844
	 Epoch : 4 | Valid Loss : 0.6116 | Valid Acc

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.608 | Test Acc : 0.6659


### bi-LSTM

In [ ]:
model_config['model_type'] = 'LSTM'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-LSTM_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.4748  Acc : 0.7333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6152 | Train Acc : 0.6495
	 Epoch : 0 | Valid Loss : 0.5472 | Valid Acc : 0.7189
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.673  Acc : 0.6333
	 Epoch : 1 | Train Loss : 0.4824 | Train Acc : 0.7651
	 Epoch : 1 | Valid Loss : 0.5653 | Valid Acc : 0.69
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.3318  Acc : 0.8
	 Saved at 2-epoch
	 Epoch : 2 | Train Loss : 0.3392 | Train Acc : 0.8553
	 Epoch : 2 | Valid Loss : 0.4323 | Valid Acc : 0.814
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.3083  Acc : 0.8667
	 Saved at 3-epoch
	 Epoch : 3 | Train Loss : 0.244 | Train Acc : 0.9031
	 Epoch : 3 | Valid Loss : 0.4001 | Valid Acc : 0.8289
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.3233  Acc : 0.8333
	 Epoch : 4 | Train Loss : 0.1508 | Train Acc : 0.9444
	 Epoc

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.414 | Test Acc : 0.8285


### bi-GRU

In [ ]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-GRU_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.5793  Acc : 0.7333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6023 | Train Acc : 0.6658
	 Epoch : 0 | Valid Loss : 0.5319 | Valid Acc : 0.7335
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.4589  Acc : 0.7333
	 Epoch : 1 | Train Loss : 0.4822 | Train Acc : 0.7669
	 Epoch : 1 | Valid Loss : 0.5393 | Valid Acc : 0.7175
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.2529  Acc : 0.9
	 Saved at 2-epoch
	 Epoch : 2 | Train Loss : 0.4126 | Train Acc : 0.8121
	 Epoch : 2 | Valid Loss : 0.4136 | Valid Acc : 0.8115
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.1481  Acc : 0.9667
	 Epoch : 3 | Train Loss : 0.2558 | Train Acc : 0.8972
	 Epoch : 3 | Valid Loss : 0.4763 | Valid Acc : 0.8091
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.1691  Acc : 0.9333
	 Epoch : 4 | Train Loss : 0.1581 | Train Acc : 0.9412
	 Epoch : 4 | Valid L

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.416 | Test Acc : 0.8117


## Inference

In [ ]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
model.load_state_dict(torch.load(f"./{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}.pt"))

<All keys matched successfully>

In [ ]:
def predict_sentiment(model, sentence):
    model.eval()
    indexed = TEXT.numericalize(TEXT.pad([TEXT.tokenize(PreProcessingText(sentence))]))
    input_data = torch.LongTensor(indexed).to(device)
    prediction = torch.sigmoid(model(input_data))
    return prediction.item()

In [ ]:
test_sentence = 'this movie is FUN'
predict_sentiment(model = model, sentence = test_sentence)

0.9888041615486145